In [25]:
import pandas as pd
import numpy as np
from itertools import combinations
import glob

In [26]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [27]:
train['title'] = train['Name'].str.split(',', expand= True)[1].str.split('.', expand = True)[0].str.strip()
train['apha_ticket'] = np.where(train['Ticket'].str.contains('[A-Za-z]') , 1, 0)
train['cabin_type'] = train['Cabin'].str[:1]
train['Pclass'] = train['Pclass'].astype(str)

In [28]:
test['title'] = test['Name'].str.split(',', expand= True)[1].str.split('.', expand = True)[0].str.strip()
test['apha_ticket'] = np.where(test['Ticket'].str.contains('[A-Za-z]') , 1, 0)
test['cabin_type'] = test['Cabin'].str[:1]
test['Pclass'] = test['Pclass'].astype(str)

In [29]:
test['Survived'] = None

In [30]:
cat_var = ['Sex', 'title', 'cabin_type', 'Embarked', 'Pclass', 'Parch', 'SibSp']

train = train.drop(['Name','Ticket', 'Cabin', 'PassengerId'] + cat_var, axis = 1)
test = test.drop(['Name','Ticket', 'Cabin', 'PassengerId'] + cat_var, axis = 1)


In [31]:
from feature_engine.imputation import MeanMedianImputer

median_imputer = MeanMedianImputer(
                   imputation_method='median',
                   variables=['Age', 'Fare']
                   )

# fit the imputer
median_imputer.fit(train)
train= median_imputer.transform(train)
test= median_imputer.transform(test)


In [33]:
feats = glob.glob('.feataz/*')

In [34]:
woe_sc = [x for x in feats if 'woesc' in x and 'train' in x]
feats_new = [x for x in feats if 'woesc' not in x and 'train' in x]

In [88]:
import random
from feature_engine.selection import DropCorrelatedFeatures
from sklearn.model_selection import train_test_split


k = 0
pred_tr, pred_ts, pred_val = [], [], []
while k <= 100:
    rand_feat = random.sample(woe_sc, 5)  + random.sample(feats_new, 10)  
    k = k + 1
    tr, ts = pd.DataFrame([]), pd.DataFrame([])
    for i in rand_feat:
        j = i.replace('train_','test_')
        feat_tr = pd.read_parquet(i)
        feat_tr = pd.read_parquet(j)
        tr = pd.concat([tr, feat_tr], axis = 1)
        ts = pd.concat([ts, feat_tr], axis = 1)

    tr = pd.concat([train, tr], axis = 1)

    ts = pd.concat([train, ts], axis = 1)

    dr = DropCorrelatedFeatures(variables=None, method='pearson', threshold=0.8)

    dr.fit(tr.drop('Survived', axis = 1))
    
    tr = tr.drop(dr.features_to_drop_, axis = 1).fillna(0)
    ts = ts.drop(dr.features_to_drop_, axis = 1).fillna(0)

    X_train, X_val, y_train, y_val = train_test_split(tr.drop('Survived', axis = 1), tr['Survived'], test_size=0.25, random_state=2107)
    
    lr= LogisticRegression(max_iter = 100000)
    
    lr.fit(X_train, y_train)
    pred_tr.append(lr.predict_proba(X_train)[:,1])
    pred_val.append(lr.predict_proba(X_val)[:,1])

    auc_val = roc_auc_score(y_val, pd.DataFrame(pred_val).transpose().mean(axis = 1)) 
    auc_train = roc_auc_score(y_train, pd.DataFrame(pred_tr).transpose().mean(axis = 1))

    print("auc train ==> ", auc_train , '\t\t', "auc validation ==> ", auc_val, '\n' )
    pred_ts.append(lr.predict_proba(ts.drop('Survived', axis = 1))[:,1])

auc train ==>  0.7229222922292229 		 auc validation ==>  0.5718832891246685 



/home/zaenal/feataz/.azenv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


auc train ==>  0.7332358235823582 		 auc validation ==>  0.6075154730327145 

auc train ==>  0.7345109510951096 		 auc validation ==>  0.5938992042440319 

auc train ==>  0.7370612061206122 		 auc validation ==>  0.6180371352785146 



/home/zaenal/feataz/.azenv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


auc train ==>  0.7373237323732373 		 auc validation ==>  0.6217506631299735 

auc train ==>  0.7370799579957996 		 auc validation ==>  0.6161803713527851 

auc train ==>  0.7362267476747675 		 auc validation ==>  0.6212201591511937 

auc train ==>  0.7354860486048604 		 auc validation ==>  0.6139699381078692 

auc train ==>  0.7368174317431744 		 auc validation ==>  0.6173297966401415 

auc train ==>  0.7364798979897988 		 auc validation ==>  0.6213085764809902 



/home/zaenal/feataz/.azenv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ValueError: could not convert string to float: 'Rare'

In [83]:
from sklearn.metrics import roc_auc_score

for i in range(len(pred_tr)):
    roc_x = roc_auc_score( tr['Survived'], pred_tr[i])
    print(roc_x)

0.7051257469721663
0.7037383227345839
0.7028648579554533
0.7035172935374259
0.6977146113614334
0.7004148957700871
0.7025160046442762
0.7046517325493454
0.699512137964827
0.7036610956656975
0.7001459325301719


In [37]:
lr = LogisticRegression()

0.7045718424780835

In [47]:
lr.predict_proba(test.drop('Survived', axis = 1))

array([[0.70737822, 0.29262178],
       [0.75154672, 0.24845328],
       [0.78827382, 0.21172618],
       [0.67755831, 0.32244169],
       [0.64520351, 0.35479649],
       [0.62583926, 0.37416074],
       [0.6921694 , 0.3078306 ],
       [0.59552981, 0.40447019],
       [0.64907677, 0.35092323],
       [0.65184493, 0.34815507],
       [0.68399196, 0.31600804],
       [0.68426985, 0.31573015],
       [0.36559454, 0.63440546],
       [0.74260989, 0.25739011],
       [0.61012131, 0.38987869],
       [0.64979804, 0.35020196],
       [0.69333827, 0.30666173],
       [0.6605047 , 0.3394953 ],
       [0.73076079, 0.26923921],
       [0.74449594, 0.25550406],
       [0.64840588, 0.35159412],
       [0.68454927, 0.31545073],
       [0.650093  , 0.349907  ],
       [0.50170858, 0.49829142],
       [0.05275976, 0.94724024],
       [0.78177256, 0.21822744],
       [0.44276275, 0.55723725],
       [0.66614252, 0.33385748],
       [0.6489286 , 0.3510714 ],
       [0.63239734, 0.36760266],
       [0.

In [77]:
 lr.fit(train.drop('Survived', axis = 1), train['Survived'])

LogisticRegression(max_iter=100000)

In [81]:
roc_auc_score( tr['Survived'],(lr.predict_proba(train.drop('Survived', axis = 1)))[:, 1])

0.6899492964347724